<a href="https://colab.research.google.com/github/yuukimiyo/sample-ginza-and-biggraph/blob/master/sample_ginza_and_biggraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リクルートのGiNZAとBiggraphを用いて萩原朔太郎の詩を分析するサンプルコード

リクルートが公開している自然言語処理ライブラリのGiNZAを用いて用意した単語間の係り受け関係を、Biggraphを用いて処理するためのサンプルコードです。

# 前処理

In [0]:
!mkdir download
!mkdir -p data/sakutarou

## 学習データを取得
青空文庫からテキストをダウンロードして解凍

In [0]:
# 月に吠える(1917)
!wget -qP download https://www.aozora.gr.jp/cards/000067/files/859_ruby_21655.zip

# 青猫(1923)
!wget -qP download https://www.aozora.gr.jp/cards/000067/files/1768_ruby_18600.zip

# 蝶を夢む(1924)
!wget -qP download https://www.aozora.gr.jp/cards/000067/files/1769_ruby_18601.zip

# 純情小曲集(1925)
!wget -qP download https://www.aozora.gr.jp/cards/000067/files/1788_ruby_18602.zip

# 氷島(1934)
!wget -qP download https://www.aozora.gr.jp/cards/000067/files/4869_ruby_14055.zip

# すべてのZipファイルを解凍
!unzip -q 'download/*.zip' -d data/sakutarou "*.txt"


5 archives were successfully processed.


## GiNZAをインストール

In [0]:
!pip -q install "https://github.com/megagonlabs/ginza/releases/download/latest/ginza-latest.tar.gz"

     |████████████████████████████████| 10.2MB 4.3MB/s 
     |████████████████████████████████| 54.6MB 824kB/s 
     |████████████████████████████████| 81kB 24.7MB/s 
     |████████████████████████████████| 70.7MB 512kB/s 
     |████████████████████████████████| 2.1MB 27.4MB/s 
     |████████████████████████████████| 3.7MB 32.4MB/s 
     |████████████████████████████████| 122kB 49.9MB/s 


***ここで「ランタイムを再起動」***

## GiNZAの係り受け関係の日本語変換用辞書の作成

In [0]:
ddic = {
    "acl": "名詞の節修飾語",
    "advcl": "副詞節修飾語",
    "advmod": "副詞修飾語",
    "amod": "形容詞修飾語",
    "appos": "同格",
    "aux": "助動詞",
    "case": "格表示",
    "cc": "等位接続詞",
    "ccomp": "補文",
    "clf": "類別詞",
    "compound": "複合名詞",
    "conj": "結合詞",
    "cop": "連結詞",
    "csubj": "主部",
    "dep": "不明な依存関係",
    "det": "限定詞",
    "discourse": "談話要素",
    "dislocated": "転置",
    "expl": "嘘辞",
    "fixed": "固定複数単語表現",
    "flat": "同格複数単語表現",
    "goeswith": "1単語分割表現",
    "iobj": "関節目的語",
    "list": "リスト表現",
    "mark": "接続詞",
    "nmod": "名詞修飾語",
    "nsubj": "主語名詞",
    "nummod": "数詞修飾語",
    "obj": "目的語",
    "obl": "斜格名詞",
    "orphan": "独立関係",
    "parataxis": "並列",
    "punct": "句読点",
    "reparandum": "単語として認識されない単語表現",
    "root": "ROOT",
    "vocative": "発声関係",
    "xcomp": "補体"
}

pdic = {
    "ADJ": "形容詞",
    "ADP": "設置詞",
    "ADV": "副詞",
    "AUX": "助動詞",
    "CCONJ": "接続詞",
    "DET": "限定詞",
    "INTJ": "間投詞",
    "NOUN": "名詞",
    "NUM": "数詞",
    "PART": "助詞",
    "PRON": "代名詞",
    "PROPN": "固有名詞",
    "PUNCT": "句読点",
    "SCONJ": "連結詞",
    "SYM": "シンボル",
    "VERB": "動詞",
    "X": "その他"
}

# メイン処理

In [0]:
import spacy
import codecs
import re
from tqdm import tqdm

In [0]:
# 対象言語セットを日本語(GiNZA)としてSpyCyを初期化
nlp = spacy.load('ja_ginza')

In [0]:
# 読み込む書籍データを作成
books = [
    # タイトル, ファイル, 読込開始行, 読込終了行
    ("月に吠える", "data/sakutarou/02tsukini_hoeru.txt", 138, 1172),
    ("青猫", "data/sakutarou/aoneko.txt", 71, 1547),
    ("蝶を夢む", "data/sakutarou/choo_yumemu.txt", 47, 1281),
    ("純情小曲集", "data/sakutarou/02junjo_shokyokushu.txt", 73, 538),
    ("氷島", "data/sakutarou/hyoto.txt", 40, 566)
]

## データのクレンジング処理

In [0]:
# クレンジング関数を作成

# 正規表現置き換えのパターン作成
ptns = []
ptns.append(re.compile(r"＃「」"))
ptns.append(re.compile(r"\\+n"))
ptns.append(re.compile(r"[\s]+"))
ptns.append(re.compile(r"※"))
ptns.append(re.compile(r"#[!-~]+"))
ptns.append(re.compile(r"［[^］]*］")) # 青空文庫注釈の削除
ptns.append(re.compile(r"《[^》]*》")) # 青空文庫ルビの削除

def cleanText(t):
    # 前後の改行等削除
    t = t.strip()
    
    # 正規表現による置き換えを実行
    for ptn in ptns:
        t = ptn.sub("", t)
    
    return t

In [0]:
# クレンジング処理を実施
lines = []
for book in books:
    with codecs.open(book[1], 'r', encoding='cp932') as f:
        for i, l in enumerate(f.readlines()):

            # 前文を削除
            if i < book[2]:
                continue;

            # あとがき等を削除
            if i > book[3]:
                break;

            l = cleanText(l)
            if len(l) > 3:
                lines.append(l)

len(lines)

3466

In [0]:
# クレンジング前
with codecs.open('./data/sakutarou/02tsukini_hoeru.txt', 'r', encoding='cp932') as f:
    for i, l in enumerate(f.readlines()):
        print(l)
        if i > 40:
            break

In [0]:
# クレンジング後
lines[0:100]

## トークナイズ／Embedding処理

### トークナイザ更新用関数

In [0]:
def filter_spans(spans):
    # Tokenizerの更新に必要なフレーズ配列を生成する
    
    # 文字列の長さ（より長いもの）、文中の位置（より後ろのもの）でソート
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result

def update_tokenizer(doc):
    # Tokenizerを固有表現/名詞区区切りに更新する
    
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)

    # 単語の区切りを、固有表現や名詞句の塊で置き換える
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)
    return doc

In [0]:
# Tokenizer更新の確認
text = "東京エリアの日本武道館"
    
# doc_tmp = nlp(text)
doc_tmp = update_tokenizer(doc_tmp)

for ent in doc_tmp:
    print('{:<30}{}'.format(ent.text, ent.dep_))

東京エリア                         nmod
の                             case
日本武道館                         ROOT


## 学習用データの作成

更新したトークナイザを利用し、単語を抽出しながら係り受け関係を取得する。

In [0]:
source_list = []
result = []
for line in tqdm(lines):
    doc = nlp(line)
    
    # トークナイザーを更新（固有表現抽出）
    doc = update_tokenizer(doc)
    
    for token in doc:
        
        # 係り元と係り先が同じ場合は無視
        if token.text == token.head.text:
            continue
        
        # 特定の係り受け関係は無視
        # if token.dep_ in ("punct", "case", "cc"):
        #     continue

        # 特定の品詞は無視
        if token.pos_ in ("AUX", "INTJ", "PUNCT", "ADP", "CCONJ", "PART", "SCONJ"):
            continue

        source_list.append(line)
        result.append("{}\t{}\t{}".format(token.lemma_, ddic[token.dep_.lower()], token.head.lemma_))
        # print(token.text, token.dep_, ddic[token.dep_.lower()], token.tag_, token.pos_, pdic[token.pos_], pdic[token.head.pos_], token.head.text, [child for child in token.children])

len(result)

100%|██████████| 3466/3466 [00:53<00:00, 65.00it/s]


11757

### 抽出した係り受け関係を保存

In [0]:
with codecs.open('data/sakutarou_train.tsv', mode='w', encoding='utf-8') as f:
    for t in result:
        f.write(t + "\n")

# BigGraph関連

In [0]:
!pip -q install torchbiggraph
!pip -q install pyflann
!pip -q install 2to3

上でインストールしたpyflannはPython3だとエラーが出たので、2to3でコードを修正する

In [0]:
!2to3 -w /usr/local/lib/python3.6/dist-packages/pyflann

### Biggraph用の設定ファイルを作成して保存

In [0]:
config_file_string = '''#!/usr/bin/env python3

def get_torchbiggraph_config():

    config = dict(
        # I/O data
        entity_path='entity',
        edge_paths=[
            'entity/sakutarou_train_partitioned'
            ],
        checkpoint_path='model',

        # Graph structure
        entities={
            'all': {'num_partitions': 1},
        },
        relations=[{
            'name': 'all_edges',
            'lhs': 'all',
            'rhs': 'all',
            'operator': 'complex_diagonal',
        }],
        dynamic_relations=True,

        # Scoring model
        dimension=400,
        global_emb=False,
        comparator='dot',

        # Training
        num_epochs=10,
        num_uniform_negs=1000,
        loss_fn='softmax',
        lr=0.1,

        # Evaluation during training
        eval_fraction=0,  # to reproduce results, we need to use all training data
    )

    return config'''

with codecs.open('conf.py', mode='w') as f:
    f.write(config_file_string)

### Biggraph用のデータ作成ツールを使用して学習用データを作成

In [0]:
!torchbiggraph_import_from_tsv --lhs-col=0 --rel-col=1 --rhs-col=2 conf.py data/sakutarou_train.tsv

Looking up relation types in the edge files...
- Found 20 relation types
- Removing the ones with fewer than 1 occurrences...
- Left with 20 relation types
- Shuffling them...
Searching for the entities in the edge files...
Entity type all:
- Found 4113 entities
- Removing the ones with fewer than 1 occurrences...
- Left with 4113 entities
- Shuffling them...
Preparing counts and dictionaries for entities and relation types:
- Writing count of entity type all and partition 0
- Writing count of dynamic relations
Preparing edge path entity/sakutarou_train_partitioned, out of the edges found in data/sakutarou_train.tsv
- Edges will be partitioned in 1 x 1 buckets.
- Processed 11757 edges in total


### 作成したデータを使用して学習処理を実施

In [0]:
!torchbiggraph_train conf.py -p edge_paths=entity/sakutarou_train_partitioned

2019-10-18 06:55:14,513   [Trainer-0] Loading entity counts...
2019-10-18 06:55:14,514   [Trainer-0] Creating workers...
2019-10-18 06:55:14,560   [Trainer-0] Initializing global model...
2019-10-18 06:55:14,684   [Trainer-0] Starting epoch 1 / 10, edge path 1 / 1, edge chunk 1 / 1
2019-10-18 06:55:14,684   [Trainer-0] Edge path: entity/sakutarou_train_partitioned
2019-10-18 06:55:14,684   [Trainer-0] still in queue: 0
2019-10-18 06:55:14,685   [Trainer-0] Swapping partitioned embeddings None ( 0 , 0 )
2019-10-18 06:55:14,685   [Trainer-0] ( 0 , 0 ): Loading entities
2019-10-18 06:55:21,379   [Trainer-0] ( 0 , 0 ): bucket 1 / 1 : Processed 11757 edges in 6.69 s ( 0.0018 M/sec ); io: 0.01 s ( 31.32 MB/sec )
2019-10-18 06:55:21,379   [Trainer-0] ( 0 , 0 ): loss:  13.5788 , violators_lhs:  439.009 , violators_rhs:  453.191 , count:  11757
2019-10-18 06:55:21,379   [Trainer-0] Swapping partitioned embeddings ( 0 , 0 ) None
2019-10-18 06:55:21,379   [Trainer-0] Writing partitioned embedding

# モデルを利用して類似単語を抽出する

In [0]:
import json
import pickle

import h5py
from pyflann import FLANN
import pandas as pd

### 結果確認用の辞書（単語ID -> 単語文字列）を作成

Biggraphの学習済みモデルは内部で単語をID化して扱っている。
ID化された出力を人が理解できる単語文字列に変換するための辞書を、Biggraphの学習用データ作成ツールが作成したjsonファイルから生成する（こういったjsonファイルは普通どの実装でも作成される）

In [0]:
with open("entity/entity_names_all_0.json") as f:
    dictionary = json.load(f)

### Biggraphによる学習済みモデルファイルを読み込む

h5形式については次など参照<br />
https://qiita.com/simonritchie/items/23db8b4cb5c590924d95

In [0]:
with h5py.File("model/embeddings_all_0.v10.h5", "r") as f:
    embeddings = f["embeddings"][:, :]

ベクトル距離を扱うためのライブラリ？、flannに読み込んだモデルデータを渡す。<br />
各用語については次など参照になるかも<br />
https://myenigma.hatenablog.com/entry/2016/04/19/214813

In [0]:
flann = FLANN()
flann.build_index(embeddings)

{'algorithm': 'kdtree',
 'branching': 32,
 'build_weight': 0.009999999776482582,
 'cb_index': 0.5,
 'centers_init': 'random',
 'checks': 32,
 'eps': 0.0,
 'iterations': 5,
 'leaf_max_size': 4,
 'log_level': 'warning',
 'memory_weight': 0.0,
 'random_seed': 997840035,
 'sample_fraction': 0.10000000149011612,
 'speedup': 0.0,
 'target_precision': 0.8999999761581421,
 'trees': 1}

flannを使って、指定した単語の類似単語を出力する関数を作成

In [0]:
def search(word_str):
    
    # 入力単語が辞書に無い場合は停止
    if word_str not in dictionary:
        print("Dictionary doesn't have {}.")
        return
    
    # 入力単語をbiggraphのモデルで使用されているIDに変換
    word_index = dictionary.index(word_str)
    
    # flannを使用して距離の近い単語を取得（しながら表示）
    for rank, result_index in enumerate(flann.nn_index(embeddings[word_index], num_neighbors=10)[0][0]):
        result_title = dictionary[result_index]
        print(rank, " ", result_title)

In [0]:
search("")

0   人
1   ところ
2   子供
3   生涯
4   私
5   夜風
6   そば
7   書生
8   火
9   通り


In [0]:
search("竹")

In [0]:
search("悠々")

In [0]:
search("妹")

0   妹
1   疱瘡
2   けた幽靈
3   女の子
4   親鳥
5   栗鼠
6   絹いと
7   日傭人
8   むぐらもち
9   齲ばみ酢


In [0]:
search("死")

In [0]:
search("死")